In [1]:
from collections import Counter, defaultdict

from estnltk import Span, Layer, Text
from estnltk.converters import text_to_json, json_to_text
from estnltk.taggers import VabamorfTagger

import json
import math
import nltk
import os
import re
import string 

In [2]:
def faili_info_salvestamine(filename):
    global keskmised
    
    andmed_name = os.path.join(target, ".".join(filename.split(".")[:-1]) + "_andmed.json")
    
    # Väljastatavate andmete sõnastik
    andmed = defaultdict(float)
    
    with open(os.path.join(source, filename), "r", encoding="UTF-8") as f:
        # Loeb failist sisu
        pure = "".join(f.readlines())
        
    # Regexiga leiab kõik potentsiaalsed emojid tekstist (kahe kooloni vaheline whitespaceita tekst)
    emojid = re.findall(":\S+?:", pure)
    # Eemaldab leitud emojide hulgast väärvasted
    wrong = [":http:", ":https:"]
    for value in wrong:
        while value in emojid:
            emojid.remove(value)
    # Eemaldab emojid tekstist, et nende sisu ei peetaks sõnadeks 
    for emoji in emojid:
        pure = pure.replace(emoji, "")

    # Otsib tekstist emotikone ja paneb need nimekirja
    emotikonid_leitud = []
    for emotikon in emotikonid:
        emotikonid_leitud.extend(re.findall(re.escape(emotikon), pure, re.IGNORECASE))
        # Eemaldab tekstist leitud emotikonid, et need sõnestamisel lahku löömisel need keskmiseid ei mõjutaks
        pure = re.sub(re.escape(emotikon), '', pure, re.IGNORECASE)
    # Vaatab tekstist eraldi emotikone, mis võivad olla ka kokkukleepumise tulemusel väärpositiivsed vasted
    # Lisaks eemaldad leitud emotikonid
    sobivad = []
    for emotikon in emotikonid_probleemsed:
        # Kui "silmad" on viimane emotikoni osa, kontrollib, et emotikoni ees ei oleks tegu tähemärgiga ehk et poleks seoses sõnaga
        if emotikon[-1] == ":":
            sobivad.extend(re.findall(re.escape(emotikon), "\n".join(re.findall("\W"+re.escape(emotikon), pure, re.IGNORECASE)), re.IGNORECASE))
            pure = re.sub("(\W)"+re.escape(emotikon), '\1', pure, re.IGNORECASE)
        # Vastasel juhul kontrollib seda emotikoni lõpust
        else:
            sobivad.extend(re.findall(re.escape(emotikon), "\n".join(re.findall(re.escape(emotikon)+"\W", pure, re.IGNORECASE)), re.IGNORECASE))
            pure = re.sub(re.escape(emotikon)+"(\W)", '\1', pure, re.IGNORECASE)
        
    # Jätab meelde emotikonide arvud ja loendid vigade kontrollimiseks
    andmed['emotikonide_arv']=len(emojid) + len(emotikonid_leitud) + len(sobivad)
        
    # Teeb morfoloogilise analüüsi nii tundmatude analüüsi oletamisega ja oletamiseta
    oletamisega = Text(pure)
        
    oletamisega.tag_layer(['words', 'sentences', 'compound_tokens'])
        
    oletamisega_morph_tagger.tag( oletamisega )
    
    # Loeb kokku lemmade arvud ja käänduvate lemmade arvud
    kõikide_lemmade_arv = 0
    ainult_käänduvate_lemmade_arv = 0
    
    for lemma, postag in zip(oletamisega.lemma, oletamisega.partofspeech):
        kõikide_lemmade_arv += 1
        # Kui tegu on käänduva lemmaga
        # Vaatab ainult esimest sõnaliiki (et välistada käändelsi verbivorme omadussõnade hulgast)
        if postag[0] in ["A", "C", "G", "H", "K", "N", "O", "P", "S", "U", "Y"]:
            ainult_käänduvate_lemmade_arv += 1

    # Loeb kokku lemmad lahutades liitsõnad osasõnadeks
    lemmas_subwords = []
    for tokens in oletamisega.root_tokens:
        lemmad = None
        # Võtab lemmade loendist esimese tõlgenduse:
        lemmad = tokens[0]
        # Vaatab iga lemmat tekstis
        for lemma in lemmad:
            # Kui kõik tähemärgid ei ole punktuatsioonimärgid
            if not all(char in string.punctuation for char in lemma):
                lemmas_subwords.append(lemma)
        
    # Võtab sõnade algvormid, ignoreerib kirjavahemärke
    lemmad = [lemma[0] for lemma in oletamisega.lemma  if not all(char in string.punctuation for char in lemma)]
        
    # Arvutab TTR-i, keskmise lemma osasõna pikkuse ja käänduvate lemmade osaarvu
    andmed['TTR'] = len(Counter(lemmad))/len(lemmad)
    andmed['lemmapikkuse_osaarv'] = sum(map(len, lemmas_subwords))/len(lemmas_subwords)
    andmed['käändsõnade_osaarv'] = ainult_käänduvate_lemmade_arv/kõikide_lemmade_arv
    
    # Arvutab tajuverbide osaarvu kõikidest verbidest ja jätab meelde
    andmed['tajuverbide_osaarv'] = tajuverbide_keskmine(oletamisega)
    
    # Loendab kokku vähemalt kolm korda korduvad tähed sõna sees
    andmed['korduvate_tähtede_arv'] = leia_korduvad_tähed(oletamisega)
    
    # Loendab kokku sõnadesse kokku kleepunud kirjavahemärkide arvu
    andmed['kokkukleepunud_kirjavahemärkide_arv'] = leia_kokkukleepunud_kirjavahemärgid(oletamisega)
    
    # Leiab verbide isikute protsendid (kui palju on 1., 2. ja 3. isik)
    verbide_isikute_protsendid = verbide_isikute_osakaalud(oletamisega)
    andmed['verbide_esimese_isiku_osaarv'] = verbide_isikute_protsendid[0]
    andmed['verbide_teise_isiku_osaarv'] = verbide_isikute_protsendid[1]
    andmed['verbide_kolmanda_isiku_osaarv'] = verbide_isikute_protsendid[2]
    
    # Leiab asesõnade isikute protsendid (kui palju on 1., 2. ja 3. isik)
    asesonade_isikute_protsendid = asesonade_isikute_osakaalud(oletamisega)
    andmed['asesõnade_esimese_isiku_osaarv'] = asesonade_isikute_protsendid[0]
    andmed['asesõnade_teise_isiku_osaarv'] = asesonade_isikute_protsendid[1]
    andmed['asesõnade_kolmanda_isiku_osaarv'] = asesonade_isikute_protsendid[2]
    
    # Leiab passiivi osakaalu ja jätab meelde
    andmed['passiivi_osakaal'] = passiivi_osakaal(oletamisega)
    
    # Leiab nud-partitsiibi osakaalu ja jätab meelde
    andmed['nud-partitsiibiga_verbide_osakaal'] = nud_osakaal(oletamisega)
    
    # Leiab kaudse kõneviisi osakaalu ja jätab meelde
    andmed['kaudse_kõneviisi_osakaal'] = vat_osakaal(oletamisega)
    
    # Leiab, kui palju tekstist on väikese algustähega või läbinisti suur
    valed_suurused = vale_tähesuurus_osakaal(oletamisega)
    andmed['puuduva_suure_algustähega'] = valed_suurused[0]
    andmed['läbinisti_suur'] = valed_suurused[1]
    
    # Leiab korduvate sõnade arvu
    andmed['korduvate_sõnade_arv'] = korduvate_sõnade_arv(oletamisega)
    
    # Leiab korduvate kokkukleepunud "juppide" arvu
    # nt silbid aga ka muud arbitraarsed kordused üle 2 tähe ja 3 korduse
    andmed['korduvate_juppide_arv'] = leia_korduvad_jupid(oletamisega)
    
    andmed['leksikonides_esinevade_osaarv'] = sõnaloendi_osaarv(oletamisega)
    
    # Kirjutab metainfo tagasi faili edasiseks skoori arvutamiseks
    with open(andmed_name, 'w', encoding="UTF-8") as fw:
        json.dump(andmed, fw, sort_keys=True, indent=4)

In [3]:
#Tajuverbide keskmise arvutamine
def tajuverbide_keskmine(oletamisega):        
    all_verbs = 0
    only_tajuverbs = 0

    for lemma, postag in zip(oletamisega.lemma, oletamisega.partofspeech):
        if "V" in postag:
            all_verbs += 1
            for lemmake in lemma:
                if lemmake in tajuverbid:
                    only_tajuverbs += 1
    # Kui tekstis ei leidu verbe, tagastab -1
    if all_verbs == 0:
        return -1
    return only_tajuverbs / float(all_verbs)

In [4]:
#Korduvate tähtede leidmine
def leia_korduvad_tähed(oletamisega):
    rx = re.compile(r'[^\d\s.:,;\(\)\[\]][.:,;\(\)\[\]][^\d\s.:,;\(\)\[\]]', re.IGNORECASE)
    rxx = rx.findall(oletamisega.text)
    return len(rxx)

In [5]:
# Korduvate (mitte kokku kleepunud) sõnade leidmine
def korduvate_sõnade_arv(oletamisega):
    rx = re.compile(r"(\b\w+\b)(\s+\1)+", re.IGNORECASE)
    rxx = rx.findall(oletamisega.text)

    return len(rxx)

In [6]:
#Kokkukleepunud kirjavahemärkide leidmine
def leia_kokkukleepunud_kirjavahemärgid(oletamisega):
    rx = re.compile(r'\D\s?[,.!?][^\d\s]')
    rxx = rx.findall(oletamisega.text)
    return len(rxx)

In [7]:
# Leiab kolm korda korduvad vähemalt kahetähelised jupid
def leia_korduvad_jupid(oletamisega):
    rx = re.compile(r"([a-zA-ZüÜõÕäÄöÖšŠžŽ])\1{3,}|([a-zA-ZüÜõÕäÄöÖšŠžŽ]{2,})\1{2,}", re.IGNORECASE)
    rxx = rx.findall(oletamisega.text)
    return len(rxx)

In [8]:
#Esimese, teise ja kolmanda isiku osakaalu leidmine verbidest
def verbide_isikute_osakaalud(oletamisega):
    # Tunnuste lõpud, EstNLTK dokumentatsioonist
    # Ei kordu omavahel
    esi_tunnused = ['sime', 'me', 'nuksime', 'nuksin', 'gem', 'ksin', 'n', 'sin', 'ksime']
    teine_tunnused = ['te', 'o', 'nuksite', 'd', 'site', 'ge', 'ksite']
    kolmas_tunnused = ['s', 'gu', 'vad', 'b']
    
    # Loeb kokku isikute kaupa ja kõik isikud kokku
    arv_koik = 0
    arv_esimene = 0
    arv_teine = 0
    arv_kolmas = 0
    
    # Vaatab iga sõna analüüsi
    for analysis in oletamisega.morph_analysis:
        # Kui pole mitmese analüüsiga
        if len(analysis.partofspeech) == 1:
            pos = analysis.partofspeech[0]
            # Kui tegu on verbiga
            if pos == "V":
                #Jätab meelde sõnalõpu
                form = analysis.form[0]
                # Vaatab, kas tunnus on loendis, ja suurendab vastavalt skoori
                if form in esi_tunnused:
                    arv_koik += 1
                    arv_esimene += 1
                    continue
                # Vaatab, kas tunnus on loendis, ja suurendab vastavalt skoori
                elif form in teine_tunnused:
                    arv_koik += 1
                    arv_teine += 1
                    continue
                # Vaatab, kas tunnus on loendis, ja suurendab vastavalt skoori
                elif form in kolmas_tunnused:
                    arv_koik += 1
                    arv_kolmas += 1
                    continue
    # Kui tekstis ei leidu isikulisi verbe, tagastab -1
    if arv_koik == 0:
        return -1, -1, -1
    # Tagastab kõik kolm osaarvu
    return arv_esimene/arv_koik, arv_teine/arv_koik, arv_kolmas/arv_koik

In [9]:
#Esimese, teise ja kolmanda isiku osakaalu leidmine asesõnadest
def asesonade_isikute_osakaalud(oletamisega):
    # Loeb kokku isikute kaupa ja kõik isikud kokku
    arv_koik = 0
    arv_esimene = 0
    arv_teine = 0
    arv_kolmas = 0
    
    # Vaatab iga sõna analüüsi
    for analysis in oletamisega.morph_analysis:
        # Kui pole mitmese analüüsiga
        if len(analysis.partofspeech) == 1:
            pos = analysis.partofspeech[0]
            # Kui tegu on asesõnaga
            if pos == "P":
                #Jätab meelde asesõna
                lemma = analysis.lemma[0]
                #Vaatab asesõna algvormi ja suurendab vastavat skoori
                if lemma == 'mina':
                    arv_koik += 1
                    arv_esimene += 1
                    continue
                #Vaatab asesõna algvormi ja suurendab vastavat skoori
                #Teietamine automaatselt sama, mis sinatamine (sina mitmuse analüüs)
                elif lemma == 'sina':
                    arv_koik += 1
                    arv_teine += 1
                    continue
                #Vaatab asesõna algvormi ja suurendab vastavat skoori
                elif lemma == 'tema':
                    arv_koik += 1
                    arv_kolmas += 1
                    continue
    # Kui tekstis ei leidu asesõnu, tagastab -1
    if arv_koik == 0:
        return -1, -1, -1
    # Tagastab kõik kolm osaarvu
    return arv_esimene/arv_koik, arv_teine/arv_koik, arv_kolmas/arv_koik

In [10]:
#Umbisikuliste verbide osakaalu leidmine tekstist
def passiivi_osakaal(oletamisega):
    # Tunnuste lõpud, EstNLTK dokumentatsioonist
    tunnused = ['takse', 'ti', 'tav', 'tuks', 'tagu', 'tama', 'tud', 'tuvat', 'tavat', 'taks', 'ta']
    
    # Loeb kokku tunnuste kaupa ja kõik kokku
    arv_koik = 0
    arv_passiiv = 0
    
    # Vaatab iga sõna analüüsi
    for analysis in oletamisega.morph_analysis:
        # Kui pole mitmese analüüsiga
        if len(analysis.partofspeech) == 1:
            pos = analysis.partofspeech[0]
            # Kui tegu on verbiga
            if pos == "V":
                #Jätab meelde sõnalõpu
                form = analysis.form[0]
                # Vaatab, kas tunnus on loendis, ja suurendab vastavalt skoori
                if form in tunnused:
                    arv_koik += 1
                    arv_passiiv += 1
                    continue
                else:
                    arv_koik += 1
        # Kui on mitmese analüüsiga
        else:
            leidub_passiivne = False
            # Vaatab kõiki analüüse
            # Kui vähemalt üks on passiivi analüüsiga
            # Märgib kogu leiduva vormi passiivseks
            for pos, form in zip(analysis.partofspeech, analysis.form):
                if pos == "V":
                    # Vaatab, kas tunnus on loendis, ja märgib, et järelikult on passiiviga
                    if form in tunnused:
                        leidub_passiivne = True
                        break
            
            if leidub_passiivne:
                arv_koik += 1
                arv_passiiv += 1
            else:
                arv_koik += 1
    # Kui tekstis ei leidu verbe, tagastab -1
    if arv_koik == 0:
        return -1
    # Tagastab osaarvu
    return arv_passiiv/arv_koik

In [11]:
#Nud-partitsiibiga verbide osakaalu leidmine tekstist
def nud_osakaal(oletamisega):
    # Tunnuste lõpud, EstNLTK dokumentatsioonist
    tunnused = ['nud']
    
    # Loeb kokku tunnuste kaupa ja kõik kokku
    arv_koik = 0
    arv_tunnus = 0
    
    # Vaatab iga sõna analüüsi
    for analysis in oletamisega.morph_analysis:
        # Kui pole mitmese analüüsiga
        if len(analysis.partofspeech) == 1:
            pos = analysis.partofspeech[0]
            # Kui tegu on verbiga
            if pos == "V":
                #Jätab meelde sõnalõpu
                form = analysis.form[0]
                # Vaatab, kas tunnus on loendis, ja suurendab vastavalt skoori
                if form in tunnused:
                    arv_koik += 1
                    arv_tunnus += 1
                    continue
                else:
                    arv_koik += 1
        # Kui on mitmese analüüsiga
        else:
            leidub_tunnusega = False
            # Vaatab kõiki analüüse
            # Kui vähemalt üks on passiivi analüüsiga
            # Märgib kogu leiduva vormi passiivseks
            for pos, form in zip(analysis.partofspeech, analysis.form):
                if pos == "V":
                    # Vaatab, kas tunnus on loendis, ja märgib, et järelikult on passiiviga
                    if form in tunnused:
                        leidub_tunnusega = True
                        break
            
            if leidub_tunnusega:
                arv_koik += 1
                arv_tunnus += 1
            else:
                arv_koik += 1
    # Kui tekstis ei leidu verbe, tagastab -1
    if arv_koik == 0:
        return -1
    # Tagastab osaarvu
    return arv_tunnus/arv_koik

In [12]:
#Vat-partitsiibiga verbide osakaalu leidmine tekstist
def vat_osakaal(oletamisega):
    # Tunnuste lõpud, EstNLTK dokumentatsioonist
    tunnused = ['vat']
    
    # Loeb kokku tunnuste kaupa ja kõik kokku
    arv_koik = 0
    arv_tunnus = 0
    
    # Vaatab iga sõna analüüsi
    for analysis in oletamisega.morph_analysis:
        # Kui pole mitmese analüüsiga
        if len(analysis.partofspeech) == 1:
            pos = analysis.partofspeech[0]
            # Kui tegu on verbiga
            if pos == "V":
                #Jätab meelde sõnalõpu
                form = analysis.form[0]
                # Vaatab, kas tunnus on loendis, ja suurendab vastavalt skoori
                if form in tunnused:
                    arv_koik += 1
                    arv_tunnus += 1
                    continue
                else:
                    arv_koik += 1
        # Kui on mitmese analüüsiga
        else:
            leidub_tunnusega = False
            # Vaatab kõiki analüüse
            # Kui vähemalt üks on passiivi analüüsiga
            # Märgib kogu leiduva vormi passiivseks
            for pos, form in zip(analysis.partofspeech, analysis.form):
                if pos == "V":
                    # Vaatab, kas tunnus on loendis, ja märgib, et järelikult on passiiviga
                    if form in tunnused:
                        leidub_tunnusega = True
                        break
            
            if leidub_tunnusega:
                arv_koik += 1
                arv_tunnus += 1
            else:
                arv_koik += 1
    # Kui tekstis ei leidu verbe, tagastab -1
    if arv_koik == 0:
        return -1
    # Tagastab osaarvu
    return arv_tunnus/arv_koik

In [13]:
def vale_tähesuurus_osakaal(oletamisega):
    vale_väike = 0
    ainult_suur = 0
    # Kõikide sõnade arv (ignoreerib kirjavahemärke)
    kõik_arv = 0

    # Vaatab iga lauset tekstis ükshaaval
    for sentence in oletamisega.sentences:
        kõik_arv += 1
        # Vaatab lause esimest sõna eraldi
        # Kas esimene sõna on täis väiketähed või täis suurtähed
        if sentence.words[0].text.islower():
            vale_väike += 1
        elif sentence.words[0].text.isupper():
            # Vaatab, et ei oleks ühe tähe suurune
            if len(sentence.words[0].text) > 1:
                ainult_suur += 1
        # Vaatab iga ülejäänud sõna lauses
        for word in sentence.words[1:]:
            # Kui sõna on ühe tähe pikkune või ainult kirjavahemärgid, jätab selle sõna vahele
            if len(word.text) == 1 or all(char in string.punctuation for char in word.text):
                continue
            kõik_arv += 1
            # Vaatab iga sõna, kas on vaid suurtähed
            if word.text.isupper():
                # Vaatab, et ei oleks lühendi analüüsiga
                lyhend = False
                if 'Y' not in word.morph_analysis.partofspeech:
                    ainult_suur += 1
                    
    # Kui tekstis ei sõnu, tagastab -1
    if kõik_arv == 0:
        return -1, -1
    return vale_väike/kõik_arv, ainult_suur/kõik_arv

In [14]:
def sõnaloendi_osaarv(oletamisega):
    kokku = len(oletamisega.words)
    loendis = 0
    
    for word in oletamisega.words:
        sona = word.text.lower()
        lemmad = word.lemma
        if sona in sõnaloend:
            loendis += 1
        else:
            for lemma in lemmad:
                if lemma.lower() in sõnaloend:
                    loendis += 1
                    break
        
    return loendis/kokku

In [15]:
sõnaloend = None

with open("../Loendid/Leksikonid/koos.txt", "r", encoding="UTF-8") as fr:
    sõnaloend = [i.strip().lower() for i in fr.readlines()]

In [16]:
tajuverbid = None

with open("../Loendid/tajuverbid/wordnet_tajuverbid.txt", "r", encoding = "utf8") as fr:
    lines = fr.readlines()
    tajuverbid = [verb.strip() for verb in lines]

In [17]:
# Loeb emotikonid sisse
emotikonid = []

for failinimi in ["wikipedia_emoticons_list.txt", "Unicode_emoticons_list.txt", "looks.wtf.txt", "unicode_emojis.txt"]:
    with open("../Loendid/emotikonid/"+failinimi, "r", encoding="UTF-8") as fr:
        for line in fr.readlines():
            # Väiketähestab
            emotikonid.append(line.strip().lower())
# Eemaldab korduvad emotikonid
emotikonid = list(set(emotikonid))

# Mõned emotikonid võivad olla ka kokkukleepumise tõttu olla väärpositiivsed (":pole")
emotikonid_probleemsed = []
with open("../Loendid/emotikonid/wikipedia_emoticons_sp.txt", "r", encoding="UTF-8") as fr:
    for line in fr.readlines():
        # Väiketähestab
        emotikonid_probleemsed.append(line.strip().lower())
# Eemaldab korduvad emotikonid 
emotikonid_probleemsed = list(set(emotikonid_probleemsed))

oletamisega_morph_tagger = VabamorfTagger(guess=True, propername=True, disambiguate=True)

In [18]:
# Algse kausta nimi, võtab sealt algsed failinimed, ei loe neid faile sisse
source = "../minikorpus/mini_tekstid/"
target = "../minikorpus/mini_andmed/"

os.makedirs(os.path.dirname(target), exist_ok=True)

with open("keskmised.json", "r", encoding = "UTF-8") as fr:
    keskmised = json.load(fr)

In [19]:
# Avab järjest kõik failid algkaustas
for file in [f for f in os.listdir(source)]:
    # Arvutab faili kohta info
    faili_info_salvestamine(file)

In [20]:
# Algse kausta nimi, võtab sealt algsed failinimed, ei loe neid faile sisse
source = "../minikorpus/lisa_tekstid/"
target = "../minikorpus/lisa_andmed/"

os.makedirs(os.path.dirname(target), exist_ok=True)

In [21]:
# Avab järjest kõik failid algkaustas
for file in [f for f in os.listdir(source)]:
    # Arvutab faili kohta info
    faili_info_salvestamine(file)